# Libraries and definitions

In [ ]:
import pandas as pd
import numpy as np
from os.path import join
from matplotlib import pyplot as plt
import sklearn
from sklearn import feature_selection
from sklearn.metrics import balanced_accuracy_score

In [ ]:
DATA_DIR = 'data'
DATA_FILE = join(DATA_DIR,'covid_data_2.csv')

In [ ]:
REF_DATE = pd.to_datetime('2022-01-01')
target_col = 'final_outcome_amp'

# Information extraction

In [ ]:
def filter_data_date(data,reference_date = REF_DATE):
    '''For each person (record_id), get only the row with the visit date closest to reference_date'''
    data['visit_date'] = pd.to_datetime(data['visit_date'])
    data = data.dropna(subset=['visit_date'],ignore_index = True)
    data = data[data['visit_date'] < reference_date]
    return data.sort_values(by=['record_id','visit_date'],ascending = [True,False]).drop_duplicates(subset=['record_id'], keep='first')#.reset_index()


def analyze_dataframe(df, target_col):
    """
    Generates a comprehensive data report for a DataFrame
    
    Args:
        df (pd.DataFrame): Input DataFrame
        target_col (str): Name of target column
        
    Returns:
        dict: Dictionary containing data characteristics
    """
    report = {
        'dataset_shape': df.shape,
        'target_balance': df[target_col].value_counts(normalize=True).to_dict(),
        'columns': {},
        'missing_data': {},
        'high_missing_cols': [],
        'date_columns': [],
        'potential_leakage': [],
        'categorical_summary': {},
        'numerical_summary': {}
    }
    
    # Check target variable
    if target_col not in df.columns:
        raise ValueError(f"Target column {target_col} not found in DataFrame")
    
    # Analyze each column
    for col in df.columns:
        if col == target_col:
            continue
            
        col_type = df[col].dtype
        unique_count = df[col].nunique()
        missing_percent = df[col].isna().mean() * 100
        
        # Store basic info
        report['columns'][col] = {
            'dtype': str(col_type),
            'unique_values': unique_count,
            'missing_percent': round(missing_percent, 2)
        }
        
        # Track high missing columns
        if missing_percent > 30:
            report['high_missing_cols'].append((col, missing_percent))
            
        # Check for potential leakage
        if any(term in col.lower() for term in ['outcome', 'result', 'target', 'final']):
            report['potential_leakage'].append(col)
            
        # Detect date columns
        if np.issubdtype(df[col].dtype, np.datetime64):
            report['date_columns'].append(col)
        elif col_type == 'object':
            try:
                pd.to_datetime(df[col])
                report['date_columns'].append(col)
            except:
                pass
                
        # Categorical analysis
        if col_type == 'object' or unique_count < 20:
            top_categories = df[col].value_counts(normalize=True).head(5).to_dict()
            report['categorical_summary'][col] = {
                'unique_count': unique_count,
                'top_categories': {k: round(v, 4) for k, v in top_categories.items()}
            }
            
        # Numerical analysis
        if np.issubdtype(col_type, np.number):
            q1 = df[col].quantile(0.25)
            q3 = df[col].quantile(0.75)
            iqr = q3 - q1
            
            num_stats = {
                'mean': round(df[col].mean(), 2),
                'median': round(df[col].median(), 2),
                'std': round(df[col].std(), 2),
                'outliers_iqr': round(((df[col] < (q1 - 1.5 * iqr)) | 
                                      (df[col] > (q3 + 1.5 * iqr))).mean() * 100, 2)
            }
            report['numerical_summary'][col] = num_stats
            
    
    # Sort high missing columns
    report['high_missing_cols'] = sorted(report['high_missing_cols'], 
                                        key=lambda x: x[1], reverse=True)
    
    return report


In [ ]:
all_data = pd.read_csv(DATA_FILE)
raw_data = filter_data_date(all_data).dropna(axis=1, how='all') #apply date filter and remove empty columns
raw_data.set_index('record_id',inplace=True)
assert (all_data['record_id'].unique() == raw_data.index).all()

In [ ]:
report = analyze_dataframe(raw_data, target_col)

# Print key insights
print(f"Dataset Shape: {report['dataset_shape']}")
print(f"\nTarget Balance: {report['target_balance']}")

print("\nHigh Missing Columns (>30%):")
for col, percent in report['high_missing_cols']:
    print(f"- {col}: {percent}% missing")
    
print("\nPotential Date Columns:")
print(report['date_columns'])

print("\nPotential Leakage Columns:")
print(report['potential_leakage'])

# Preprocess data

In [ ]:
y = raw_data.pop('final_outcome_amp')
raw_data.head()

In [ ]:
raw_data['diabetes'].dropna().unique()

## Convert variables

In [ ]:
def scale(x:pd.Series):
    if SCALE: return (x - x.mean()) / x.std()
    else: return x

In [ ]:
def scale(data):
    return data
    return (data - data.mean())/data.std()
def preprocess_data(df):
    data = df.copy().drop(columns=['Unnamed: 0', 'visit_date'])
    data['index_date'] = (REF_DATE - pd.to_datetime(data['index_date'], format='%Y-%m-%d')).dt.days; 
    data['study'] = data['study'].fillna(data['study'].mode()[0]).map({data['study'].dropna().unique()[0]:0, data['study'].dropna().unique()[1]:1})
    data['timepoint'] = data['timepoint'].fillna(data['timepoint'].mode()[0]).map({data['timepoint'].dropna().unique()[0]:0, data['timepoint'].dropna().unique()[1]:1})
    data = pd.get_dummies(data, columns=['pop_sample'], dtype=int)
    data['age'] = scale(data['age'])#normalize age?
    data['sex'] = data['sex'].fillna(data['sex'].mode()[0]).map({data['sex'].dropna().unique()[0]:0, data['sex'].dropna().unique()[1]:1})
    data['bmi'] = scale(data['bmi']).fillna(data['bmi'].mean())
    data['smoking'] = data['smoking'].fillna(data['smoking'].mode()[0]).map({'Non-smoker': 0, 'Ex-smoker': 1, 'Smoker': 2})
    data['comorbidity'] = data['comorbidity'].fillna(data['comorbidity'].mode()[0]).map({data['comorbidity'].dropna().unique()[0]:0, data['comorbidity'].dropna().unique()[1]:1})
    data['hypertension'] = data['hypertension'].fillna(data['hypertension'].mode()[0]).map({data['hypertension'].dropna().unique()[0]:0, data['hypertension'].dropna().unique()[1]:1})
    data['diabetes'] = data['diabetes'].fillna(data['diabetes'].mode()[0]).map({data['diabetes'].dropna().unique()[0]:0, data['diabetes'].dropna().unique()[1]:1})
    data['cvd'] = data['cvd'].fillna(data['cvd'].mode()[0]).map({data['cvd'].dropna().unique()[0]:0, data['cvd'].dropna().unique()[1]:1})
    data['respiratory'] = data['respiratory'].fillna(data['respiratory'].mode()[0]).map({data['respiratory'].dropna().unique()[0]:0, data['respiratory'].dropna().unique()[1]:1})
    data['ckd'] = data['ckd'].fillna(data['ckd'].mode()[0]).map({data['ckd'].dropna().unique()[0]:0, data['ckd'].dropna().unique()[1]:1})
    data['cancer'] = data['cancer'].fillna(data['cancer'].mode()[0]).map({data['cancer'].dropna().unique()[0]:0, data['cancer'].dropna().unique()[1]:1})
    data['immune_supp'] = data['immune_supp'].fillna(data['immune_supp'].mode()[0]).map({data['immune_supp'].dropna().unique()[0]:0, data['immune_supp'].dropna().unique()[1]:1})
    data['income_3l'] = data['income_3l'].fillna(data['income_3l'].mode()[0]).map({"<6'000 CHF": 0, "6'000 - 12'000 CHF": 1, ">12'000 CHF": 2})
    data = pd.get_dummies(data, columns=['employment_4l'], dtype=int)
    data['education_4l'] = data['education_4l'].fillna(data['education_4l'].mode()[0]).map({
        'None or mandatory school': 0,
        'Vocational training or specialized baccalaureate': 1,
        'Higher technical school or college': 2,
        'University': 3
    })
    data['nationality'] = data['nationality'].fillna(data['nationality'].mode()[0]).map({'Non-Swiss':0,'Swiss':1})
    data['summary_bl_behaviour'] = scale(data['summary_bl_behaviour'])
    data['symp_init'] = data['symp_init'].fillna(data['symp_init'].mode()[0]).map({"No":0, "Yes":1})
    data['symp_count_init_3l'] = data['symp_count_init_3l'].fillna(data['symp_count_init_3l'].mode()[0]).map({
        'Asymptomatic': 0,
        '1-5 symptoms': 1,
        '≥6 symptoms': 2
    })
    data['symp_sev_init_3l'] = data['symp_sev_init_3l'].fillna(data['symp_sev_init_3l'].mode()[0]).map({"Mild to moderate":0, "Severe to very severe":1})
    data['hosp_2wks'] = data['hosp_2wks'].fillna(data['hosp_2wks'].mode()[0]).map({"No":0, "Yes":1})
    data['icu_2wks'] = data['icu_2wks'].fillna(data['icu_2wks'].mode()[0]).map({"No":0, "Yes":1}) #can discard?
    data['seropos_at_bl'] = data['seropos_at_bl'].fillna(data['seropos_at_bl'].mode()[0]).map({"No":0, "Yes":1})
    data['prior_pos_pcr'] = data['prior_pos_pcr'].fillna(data['prior_pos_pcr'].mode()[0]).map({"No":0, "Yes":1})
    data['prior_exposure'] = data['prior_exposure'].fillna(data['prior_exposure'].mode()[0]).map({"No":0, "Yes":1})
    data['first_exposure_date'] = (REF_DATE - pd.to_datetime(data['first_exposure_date'], format='%Y-%m-%d')).dt.days; 
    data['first_exposure_date'] = data['first_exposure_date'].fillna(-100)
    data['first_exposure'] = data['first_exposure'].fillna(data['first_exposure'].mode()[0]).map({data['first_exposure'].dropna().unique()[0]:0, data['first_exposure'].dropna().unique()[1]:1})
    data= pd.get_dummies(data,columns=['vaccine_type_1'], dummy_na=0, dtype=int)#data['vaccine_type_1'].map({"Moderna":0, "Pfizer/BioNTech":1, "Other":0.5}).fillna(-1) 
    data = pd.get_dummies(data,columns=['vaccine_type_2'], dummy_na=0, dtype=int)#data['vaccine_type_2'].map({"Moderna":0, "Pfizer/BioNTech":1, "Other":0.5}).fillna(-1)
    data = pd.get_dummies(data,columns=['vaccine_type_3'], dummy_na=0, dtype=int)#data['vaccine_type_3'].map({"Moderna":0, "Pfizer/BioNTech":1, "Other":0.5}).fillna(-1)
    data = pd.get_dummies(data,columns=['vaccine_type_4'], dummy_na=0, dtype=int)#data['vaccine_type_4'].map({"Moderna":0, "Pfizer/BioNTech":1, "Other":0.5}).fillna(-1) 
    data['pcr_pos_date_1'] = (REF_DATE - pd.to_datetime(data['pcr_pos_date_1'], format='%Y-%m-%d')).dt.days.fillna(-100); 
    data['pcr_pos_date_1'] = data['pcr_pos_date_1'].fillna(data['pcr_pos_date_1'].mean())
    data['pcr_pos_date_2'] = (REF_DATE - pd.to_datetime(data['pcr_pos_date_2'], format='%Y-%m-%d')).dt.days.fillna(-100)
    data['pcr_pos_sev_1'] = data['pcr_pos_sev_1'].fillna(data['pcr_pos_sev_1'].mode()[0]).map({"Very mild": 1, "Mild": 2, "Moderate": 3, "Severe": 4})
    data['ab_chuv_iga_ratio'] = scale(data['ab_chuv_iga_ratio'])
    data['ab_chuv_iga_result'] = data['ab_chuv_iga_result'].fillna(data['ab_chuv_iga_result'].mode()[0]).map({data['ab_chuv_iga_result'].dropna().unique()[0]:0, data['ab_chuv_iga_result'].dropna().unique()[1]:1})
    data['ab_chuv_igg_s_ratio'] = scale(data['ab_chuv_igg_s_ratio'])
    data['ab_chuv_igg_s_result'] = data['ab_chuv_igg_s_result'].fillna(data['ab_chuv_igg_s_result'].mode()[0]).map({data['ab_chuv_igg_s_result'].dropna().unique()[0]:0, data['ab_chuv_igg_s_result'].dropna().unique()[1]:1})
    data['ab_chuv_igg_n_ratio'] = scale(data['ab_chuv_igg_n_ratio'])
    data['ab_chuv_igg_n_result'] = data['ab_chuv_igg_n_result'].fillna(data['ab_chuv_igg_n_result'].mode()[0]).map({data['ab_chuv_igg_n_result'].dropna().unique()[0]:0, data['ab_chuv_igg_n_result'].dropna().unique()[1]:1})
    data['last_antibody_before_omicron_iga_logratio'] = scale(data['last_antibody_before_omicron_iga_logratio'])
    data['ab_chuv_iga_logratio'] = scale(data['ab_chuv_iga_logratio'])
    data['ab_chuv_igg_s_logratio'] = scale(data['ab_chuv_igg_s_logratio'])
    data['ab_chuv_igg_n_logratio'] = scale(data['ab_chuv_igg_n_logratio'])
    
    # Drop the ratio columns
    data.drop(columns=['ab_chuv_iga_ratio', 'ab_chuv_igg_s_ratio', 'ab_chuv_igg_n_ratio'], inplace=True)
    data['last_antibody_before_omicron_igg_n_logratio'] = scale(data['last_antibody_before_omicron_igg_n_logratio'])
    data['last_antibody_before_omicron_igg_s_logratio'] = scale(data['last_antibody_before_omicron_igg_s_logratio'])
    data['vaccine_date_1'] = (REF_DATE - pd.to_datetime(data['vaccine_date_1'], format='%Y-%m-%d')).dt.days; 
    data['vaccine_date_1'] = data['vaccine_date_1'].fillna(-100)
    data['vaccine_date_2'] = (REF_DATE - pd.to_datetime(data['vaccine_date_2'], format='%Y-%m-%d')).dt.days
    data['vaccine_date_2'] = data['vaccine_date_2'].fillna(-100); 
    data['vaccine_date_3'] = (REF_DATE - pd.to_datetime(data['vaccine_date_3'], format='%Y-%m-%d')).dt.days; 
    data['vaccine_date_3'] = data['vaccine_date_3'].fillna(-100)
    data['vaccine_date_4'] = (REF_DATE - pd.to_datetime(data['vaccine_date_4'], format='%Y-%m-%d')).dt.days; 
    data['vaccine_date_4'] = data['vaccine_date_4'].fillna(-100)
    data['prior_hyg'] = data['prior_hyg'].fillna(data['prior_hyg'].mode()[0])
    data['prior_dist'] = data['prior_dist'].fillna(data['prior_dist'].mode()[0])
    data['prior_mask_mand'] = data['prior_mask_mand'].fillna(data['prior_mask_mand'].mode()[0])
    data["most_recent_vaccine"] = raw_data[['vaccine_date_4', 'vaccine_date_3', 'vaccine_date_2', 'vaccine_date_1']].bfill(axis=1).iloc[:, 0]
    data["most_recent_vaccine"] = (REF_DATE - pd.to_datetime(data["most_recent_vaccine"], format='%Y-%m-%d')).dt.days.fillna(-100)
    data['num_vaccines'] = raw_data[['vaccine_date_1', 'vaccine_date_2', 'vaccine_date_3', 'vaccine_date_4']].notna().sum(axis=1)
    
    return data

In [ ]:
data = preprocess_data(raw_data)

In [ ]:
print(f"Final dataset shape: {data.shape}")
print("\nMissing values:", data.isna().sum().sum())
print("Column dtypes:\n", data.dtypes.value_counts())
print("\nTarget distribution:\n", y.value_counts(normalize=True))

In [ ]:
density = False
plt.hist(data['last_antibody_before_omicron_igg_n_logratio'][y==1],histtype='step', bins = 'auto', color = 'blue', density=density, label = 'Positive')
plt.hist(data['last_antibody_before_omicron_igg_n_logratio'][y==1],histtype='stepfilled', bins = 'auto', alpha = 0.4, color='blue', density=density)
plt.hist(data['last_antibody_before_omicron_igg_n_logratio'][y==0],histtype='step', bins = 'auto', color = 'red', density=density, label='Negative')
plt.hist(data['last_antibody_before_omicron_igg_n_logratio'][y==0],histtype='stepfilled', bins = 'auto', alpha = 0.4, color='red', density=density)
plt.legend()
#plt.xlim(0,10)
plt.title('last_antibody_before_omicron_igg_n_logratio')
plt.show()


In [ ]:
density = False
plt.hist(data['ab_chuv_igg_n_logratio'][y==1],histtype='step', bins = 'auto', color = 'blue', density=density, label = 'Positive')
plt.hist(data['ab_chuv_igg_n_logratio'][y==1],histtype='stepfilled', bins = 'auto', alpha = 0.4, color='blue', density=density)
plt.hist(data['ab_chuv_igg_n_logratio'][y==0],histtype='step', bins = 'auto', color = 'red', density=density, label='Negative')
plt.hist(data['ab_chuv_igg_n_logratio'][y==0],histtype='stepfilled', bins = 'auto', alpha = 0.4, color='red', density=density)
plt.legend()
plt.xlim(0,10)
plt.show()


In [ ]:
density = False
plt.hist(data['ab_chuv_igg_s_logratio'][y==1],histtype='step', bins = 'auto', color = 'blue', density=density, label = 'Positive')
plt.hist(data['ab_chuv_igg_s_logratio'][y==1],histtype='stepfilled', bins = 'auto', alpha = 0.4, color='blue', density=density)
plt.hist(data['ab_chuv_igg_s_logratio'][y==0],histtype='step', bins = 'auto', color = 'red', density=density, label='Negative')
plt.hist(data['ab_chuv_igg_s_logratio'][y==0],histtype='stepfilled', bins = 'auto', alpha = 0.4, color='red', density=density)
plt.legend()
#plt.xlim(0,10)
plt.title('last_antibody_before_omicron_igg_s_logratio')
plt.show()

In [ ]:
raw_data["last_antibody_before_omicron_iga_logratio"].isna().sum()

In [ ]:
density = True
plt.hist(data['last_antibody_before_omicron_igg_s_logratio'][y==1],histtype='step', bins = 'auto', color = 'blue', density=density, label = 'Positive')
plt.hist(data['last_antibody_before_omicron_igg_s_logratio'][y==1],histtype='stepfilled', bins = 'auto', alpha = 0.4, color='blue', density=density)
plt.hist(data['last_antibody_before_omicron_igg_s_logratio'][y==0],histtype='step', bins = 'auto', color = 'red', density=density, label='Negative')
plt.hist(data['last_antibody_before_omicron_igg_s_logratio'][y==0],histtype='stepfilled', bins = 'auto', alpha = 0.4, color='red', density=density)
plt.legend()
#plt.xlim(0,10)
plt.show()

In [ ]:
density = True
plt.hist(data['last_antibody_before_omicron_igg_n_logratio'][y==1],histtype='step', bins = 'auto', color = 'blue', density=density, label = 'Positive')
plt.hist(data['last_antibody_before_omicron_igg_n_logratio'][y==1],histtype='stepfilled', bins = 'auto', alpha = 0.4, color='blue', density=density)
plt.hist(data['last_antibody_before_omicron_igg_n_logratio'][y==0],histtype='step', bins = 'auto', color = 'red', density=density, label='Negative')
plt.hist(data['last_antibody_before_omicron_igg_n_logratio'][y==0],histtype='stepfilled', bins = 'auto', alpha = 0.4, color='red', density=density)
plt.legend()
#plt.xlim(0,10)
plt.show()

In [ ]:
y_pred = data['last_antibody_before_omicron_igg_n_logratio'] < 0.5
accuracy = (y_pred == y).mean()
balanced_accuracy = balanced_accuracy_score(y, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print(f"Balanced Accuracy: {balanced_accuracy:.2f}")

In [ ]:
density = False
plt.hist(data['last_antibody_before_omicron_iga_logratio'][y==1],histtype='step', bins = 'auto', color = 'blue', density=density, label = 'Positive')
plt.hist(data['last_antibody_before_omicron_iga_logratio'][y==1],histtype='stepfilled', bins = 'auto', alpha = 0.4, color='blue', density=density)
plt.hist(data['last_antibody_before_omicron_iga_logratio'][y==0],histtype='step', bins = 'auto', color = 'red', density=density, label='Negative')
plt.hist(data['last_antibody_before_omicron_iga_logratio'][y==0],histtype='stepfilled', bins = 'auto', alpha = 0.4, color='red', density=density)
plt.legend()
#plt.xlim(0,10)
plt.title('last_antibody_before_omicron_iga_logratio')
plt.show()

In [ ]:
y_pred = data['last_antibody_before_omicron_iga_logratio'] < 4
accuracy = (y_pred == y).mean()
balanced_accuracy = balanced_accuracy_score(y, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print(f"Balanced Accuracy: {balanced_accuracy:.2f}")

In [ ]:
density = False
plt.hist(data['study'][y==1],histtype='step', bins = 'auto', color = 'blue', density=density, label = 'Positive')
plt.hist(data['study'][y==1],histtype='stepfilled', bins = 'auto', alpha = 0.4, color='blue', density=density)
plt.hist(data['study'][y==0],histtype='step', bins = 'auto', color = 'red', density=density, label='Negative')
plt.hist(data['study'][y==0],histtype='stepfilled', bins = 'auto', alpha = 0.4, color='red', density=density)
plt.legend()
plt.show()

In [ ]:
density = False#True
plt.hist(data['age'][y==1],histtype='step', bins = 'auto', color = 'blue', density=density, label = 'Positive')
plt.hist(data['age'][y==1],histtype='stepfilled', bins = 'auto', alpha = 0.4, color='blue', density=density)
plt.hist(data['age'][y==0],histtype='step', bins = 'auto', color = 'red', density=density, label='Negative')
plt.hist(data['age'][y==0],histtype='stepfilled', bins = 'auto', alpha = 0.4, color='red', density=density)
plt.legend()
plt.title('age')
plt.show()

In [ ]:
density = False#True
plt.hist(data['vaccine_date_3'][y==1],histtype='step', bins = 'auto', color = 'blue', density=density, label = 'Positive')
plt.hist(data['vaccine_date_3'][y==1],histtype='stepfilled', bins = 'auto', alpha = 0.4, color='blue', density=density)
plt.hist(data['vaccine_date_3'][y==0],histtype='step', bins = 'auto', color = 'red', density=density, label='Negative')
plt.hist(data['vaccine_date_3'][y==0],histtype='stepfilled', bins = 'auto', alpha = 0.4, color='red', density=density)
plt.legend()
plt.show()

In [ ]:
data['pcr_pos_date_1']

In [ ]:
most_recent_vaccine = raw_data[['vaccine_date_4', 'vaccine_date_3', 'vaccine_date_2', 'vaccine_date_1']].bfill(axis=1).iloc[:, 0]
most_recent_vaccine = (REF_DATE - pd.to_datetime(most_recent_vaccine, format='%Y-%m-%d')).dt.days.fillna(-100)

most_recent_infection = raw_data[['pcr_pos_date_2', 'pcr_pos_date_1']].bfill(axis=1).iloc[:, 0]
most_recent_infection = (REF_DATE - pd.to_datetime(most_recent_infection, format='%Y-%m-%d')).dt.days.fillna(-100)

most_recent_contact = raw_data[['pcr_pos_date_2', 'pcr_pos_date_1', 'vaccine_date_4', 'vaccine_date_3', 'vaccine_date_2', 'vaccine_date_1']].bfill(axis=1).iloc[:, 0]
most_recent_contact = (REF_DATE - pd.to_datetime(most_recent_contact, format='%Y-%m-%d')).dt.days.fillna(-100)

data['most_recent_vaccine'] = most_recent_vaccine
data['most_recent_infection'] = most_recent_infection
data['most_recent_contact'] = most_recent_contact

delta_t = (pd.to_datetime(raw_data['pcr_pos_date_1'], format='%Y-%m-%d') - pd.to_datetime(raw_data['vaccine_date_1'], format='%Y-%m-%d')).dt.days.fillna(500)
data['delta_t'] = delta_t


In [ ]:
pcr_pos_date = (REF_DATE - pd.to_datetime(raw_data['pcr_pos_date_1'], format='%Y-%m-%d')).dt.days.fillna(-100)

In [ ]:
raw_data['first_exposure'].value_counts()

In [ ]:
density = False#True
bins = 50
plt.hist(data['delta_t'][y==1],histtype='step', bins = bins, color = 'blue', density=density, label = 'Positive')
plt.hist(data['delta_t'][y==1],histtype='stepfilled', bins = bins, alpha = 0.4, color='blue', density=density)
plt.hist(data['delta_t'][y==0],histtype='step', bins = bins, color = 'red', density=density, label='Negative')
plt.hist(data['delta_t'][y==0],histtype='stepfilled', bins = bins, alpha = 0.4, color='red', density=density)
plt.legend()
plt.show()

In [ ]:
y_pred = data['delta_t'] < -250
accuracy = (y_pred == y).mean()
balanced_accuracy = balanced_accuracy_score(y, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print(f"Balanced Accuracy: {balanced_accuracy:.2f}")

In [ ]:
density = False#True
plt.hist(data['index_date'][y==1],histtype='step', bins = 'auto', color = 'blue', density=density, label = 'Positive')
plt.hist(data['index_date'][y==1],histtype='stepfilled', bins = 'auto', alpha = 0.4, color='blue', density=density)
plt.hist(data['index_date'][y==0],histtype='step', bins = 'auto', color = 'red', density=density, label='Negative')
plt.hist(data['index_date'][y==0],histtype='stepfilled', bins = 'auto', alpha = 0.4, color='red', density=density)
plt.legend()
plt.show()

In [ ]:
density = False#True
plt.hist(data['vaccine_date_3'][y==1],histtype='step', bins = 'auto', color = 'blue', density=density, label = 'Positive')
plt.hist(data['vaccine_date_3'][y==1],histtype='stepfilled', bins = 'auto', alpha = 0.4, color='blue', density=density)
plt.hist(data['vaccine_date_3'][y==0],histtype='step', bins = 'auto', color = 'red', density=density, label='Negative')
plt.hist(data['vaccine_date_3'][y==0],histtype='stepfilled', bins = 'auto', alpha = 0.4, color='red', density=density)
plt.legend()
plt.show()

In [ ]:
raw_data[raw_data['study']=='ZVAC']['pcr_pos_date_1']

In [ ]:
plt.scatter(data['age'], raw_data['employment_4l'], c=y, cmap='coolwarm', alpha=0.5)

In [ ]:
columns_to_check = ['comorbidity', 'hypertension', 'diabetes', 'cvd', 'respiratory', 'ckd', 'cancer', 'immune_supp']

for column in columns_to_check:
    print(f"Value counts for {column}:")
    print(raw_data[column].value_counts())
    print("\n")

In [ ]:
density = False#True
plt.hist(data['cancer'][y==1],histtype='step', bins = 'auto', color = 'blue', density=density, label = 'Positive')
plt.hist(data['cancer'][y==1],histtype='stepfilled', bins = 'auto', alpha = 0.4, color='blue', density=density)
plt.hist(data['cancer'][y==0],histtype='step', bins = 'auto', color = 'red', density=density, label='Negative')
plt.hist(data['cancer'][y==0],histtype='stepfilled', bins = 'auto', alpha = 0.4, color='red', density=density)
plt.legend()
plt.show()

In [ ]:
from scipy.stats import linregress

avg_vaccines_per_age = data.groupby('age')['num_vaccines'].mean()
slope, intercept, r_value, p_value, std_err = linregress(avg_vaccines_per_age.index, avg_vaccines_per_age.values)

plt.plot(avg_vaccines_per_age.index, avg_vaccines_per_age.values, marker='o', label='Average Vaccines')
plt.plot(avg_vaccines_per_age.index, intercept + slope * avg_vaccines_per_age.index, linestyle='--', label='Linear Regression')
plt.xlabel('Age')
plt.ylabel('Average Number of Vaccines')
plt.title('Average Number of Vaccines per Age')
plt.legend()
plt.show()

In [ ]:
from scipy.stats import linregress

avg_vaccines_per_time = data.groupby('most_recent_contact')['ab_chuv_igg_n_logratio'].mean()
slope, intercept, r_value, p_value, std_err = linregress(avg_vaccines_per_time.index, avg_vaccines_per_time.values)

plt.plot(avg_vaccines_per_time.index, avg_vaccines_per_time.values, marker='o', label='Average IGG Logratio')
plt.plot(avg_vaccines_per_time.index, intercept + slope * avg_vaccines_per_time.index, linestyle='--', label='Linear Regression')
plt.xlabel('Time Since Last Exposure (days)')
plt.ylabel('Average IGG Logratio')
plt.title('Average IGG_N Logratio vs. Time Since Last Contact')
plt.legend()
plt.show()

In [ ]:
from scipy.stats import linregress

avg_vaccines_per_age = data.groupby('age')['num_vaccines'].mean()
slope, intercept, r_value, p_value, std_err = linregress(avg_vaccines_per_age.index, avg_vaccines_per_age.values)

plt.plot(avg_vaccines_per_age.index, avg_vaccines_per_age.values, marker='o', label='Average Vaccines')
plt.plot(avg_vaccines_per_age.index, intercept + slope * avg_vaccines_per_age.index, linestyle='--', label='Linear Regression')
plt.xlabel('Age')
plt.ylabel('Average Number of Vaccines')
plt.title('Average Number of Vaccines per Age')
plt.legend()
plt.show()

In [ ]:
density = False#True
plt.hist(most_recent_vaccine[y==1],histtype='step', bins = 'auto', color = 'blue', density=density, label = 'Positive')
plt.hist(most_recent_vaccine[y==1],histtype='stepfilled', bins = 'auto', alpha = 0.4, color='blue', density=density)
plt.hist(most_recent_vaccine[y==0],histtype='step', bins = 'auto', color = 'red', density=density, label='Negative')
plt.hist(most_recent_vaccine[y==0],histtype='stepfilled', bins = 'auto', alpha = 0.4, color='red', density=density)
plt.legend()
plt.title('most recent vaccine')
plt.show()

In [ ]:
y_pred = (most_recent_vaccine < 0) | ((most_recent_vaccine > 115) & (most_recent_vaccine <210))
accuracy = (y_pred == y).mean()
balanced_accuracy = balanced_accuracy_score(y, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print(f"Balanced Accuracy: {balanced_accuracy:.2f}")

In [ ]:
density = False#True
plt.hist(most_recent_contact[y==1],histtype='step', bins = 'auto', color = 'blue', density=density, label = 'Positive')
plt.hist(most_recent_contact[y==1],histtype='stepfilled', bins = 'auto', alpha = 0.4, color='blue', density=density)
plt.hist(most_recent_contact[y==0],histtype='step', bins = 'auto', color = 'red', density=density, label='Negative')
plt.hist(most_recent_contact[y==0],histtype='stepfilled', bins = 'auto', alpha = 0.4, color='red', density=density)
plt.legend()
plt.title('Most recent exposure')
plt.show()

In [ ]:
density = False#True
bins = 40
plt.hist(data['first_exposure_date'][y==1],histtype='step', bins = bins, color = 'blue', density=density, label = 'Positive')
plt.hist(data['first_exposure_date'][y==1],histtype='stepfilled', bins = bins, alpha = 0.4, color='blue', density=density)
plt.hist(data['first_exposure_date'][y==0],histtype='step', bins = bins, color = 'red', density=density, label='Negative')
plt.hist(data['first_exposure_date'][y==0],histtype='stepfilled', bins = bins, alpha = 0.4, color='red', density=density)
plt.legend()
plt.show()

In [ ]:
density = False#True
bins = 40
plt.hist(data['bmi'][y==1],histtype='step', bins = bins, color = 'blue', density=density, label = 'Positive')
plt.hist(data['bmi'][y==1],histtype='stepfilled', bins = bins, alpha = 0.4, color='blue', density=density)
plt.hist(data['bmi'][y==0],histtype='step', bins = bins, color = 'red', density=density, label='Negative')
plt.hist(data['bmi'][y==0],histtype='stepfilled', bins = bins, alpha = 0.4, color='red', density=density)
plt.legend()
plt.show()

In [ ]:
density = False#True
bins = 40
plt.hist(data['age'][y==1],histtype='step', bins = bins, color = 'blue', density=density, label = 'Positive')
plt.hist(data['age'][y==1],histtype='stepfilled', bins = bins, alpha = 0.4, color='blue', density=density)
plt.hist(data['age'][y==0],histtype='step', bins = bins, color = 'red', density=density, label='Negative')
plt.hist(data['age'][y==0],histtype='stepfilled', bins = bins, alpha = 0.4, color='red', density=density)
plt.legend()
plt.show()

In [ ]:
density = False#True
plt.hist(most_recent_infection[y==1],histtype='step', bins = 'auto', color = 'blue', density=density, label = 'Positive')
plt.hist(most_recent_infection[y==1],histtype='stepfilled', bins = 'auto', alpha = 0.4, color='blue', density=density)
plt.hist(most_recent_infection[y==0],histtype='step', bins = 'auto', color = 'red', density=density, label='Negative')
plt.hist(most_recent_infection[y==0],histtype='stepfilled', bins = 'auto', alpha = 0.4, color='red', density=density)
plt.legend()
plt.show()

In [ ]:
density = False#True
plt.hist(pcr_pos_date[y==1],histtype='step', bins = 'auto', color = 'blue', density=density, label = 'Positive')
plt.hist(pcr_pos_date[y==1],histtype='stepfilled', bins = 'auto', alpha = 0.4, color='blue', density=density)
plt.hist(pcr_pos_date[y==0],histtype='step', bins = 'auto', color = 'red', density=density, label='Negative')
plt.hist(pcr_pos_date[y==0],histtype='stepfilled', bins = 'auto', alpha = 0.4, color='red', density=density)
plt.legend()
plt.show()

In [ ]:
correlations = data.corrwith(y).abs().sort_values()
print(correlations)
correlations.plot(kind='bar', figsize=(12, 6))
plt.title('Absolute Correlation of each feature with the target variable')
plt.xlabel('Features')
plt.ylabel('Absolute Correlation coefficient')
plt.show()

In [ ]:
from sklearn.feature_selection import mutual_info_classif

# Calculate mutual information
mi = mutual_info_classif(data, y, discrete_features='auto')
mi_series = pd.Series(mi, index=data.columns).sort_values()

# Plot mutual information
mi_series.plot(kind='barh', figsize=(6, 12))
plt.title('Mutual Information of each feature with the target variable')
plt.ylabel('Features')
plt.xlabel('Mutual Information')
plt.show()

In [ ]:
raw_data['pcr_pos_date_1']


In [ ]:
raw_data['prior_exposure'].value_counts()

In [ ]:
spearman_corr = data.corrwith(y, method='spearman').abs().sort_values()
print(spearman_corr)
spearman_corr.plot(kind='bar', figsize=(12, 6))
plt.title('Absolute Spearman Correlation of each feature with the target variable')
plt.xlabel('Features')
plt.ylabel('Absolute Spearman Correlation coefficient')
plt.show()

In [ ]:
zsac = data[data['study']==0]
zvac = data[data['study']==1]

In [ ]:
density = False#True
plt.hist(zvac['pcr_pos_date_1'][y==1],histtype='step', bins = 'auto', color = 'blue', density=density, label = 'Positive')
plt.hist(zvac['pcr_pos_date_1'][y==1],histtype='stepfilled', bins = 'auto', alpha = 0.4, color='blue', density=density)
plt.hist(zvac['pcr_pos_date_1'][y==0],histtype='step', bins = 'auto', color = 'red', density=density, label='Negative')
plt.hist(zvac['pcr_pos_date_1'][y==0],histtype='stepfilled', bins = 'auto', alpha = 0.4, color='red', density=density)
plt.legend()
plt.show()

In [ ]:
raw_data['first_exposure_date'].isna().sum()

In [ ]:
density = False#True
plt.hist(zvac['first_exposure_date'][y==1],histtype='step', bins = 'auto', color = 'blue', density=density, label = 'Positive')
plt.hist(zvac['first_exposure_date'][y==1],histtype='stepfilled', bins = 'auto', alpha = 0.4, color='blue', density=density)
plt.hist(zvac['first_exposure_date'][y==0],histtype='step', bins = 'auto', color = 'red', density=density, label='Negative')
plt.hist(zvac['first_exposure_date'][y==0],histtype='stepfilled', bins = 'auto', alpha = 0.4, color='red', density=density)
plt.legend()
plt.show()

In [ ]:
y_pred = data['first_exposure_date'] > 480
accuracy = (y_pred == y).mean()
balanced_accuracy = balanced_accuracy_score(y, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print(f"Balanced Accuracy: {balanced_accuracy:.2f}")

In [ ]:
density = False  # True
bins = 50

# Calculate bins based on the first histogram
hist_data, bin_edges = np.histogram(zsac['pcr_pos_date_1'][y == 1], bins=bins, density=density)

plt.hist(zsac['pcr_pos_date_1'][y == 1], histtype='step', bins=bin_edges, color='blue', density=density, label='Positive')
plt.hist(zsac['pcr_pos_date_1'][y == 1], histtype='stepfilled', bins=bin_edges, alpha=0.4, color='blue', density=density)
plt.hist(zsac['pcr_pos_date_1'][y == 0], histtype='step', bins=bin_edges, color='red', density=density, label='Negative')
plt.hist(zsac['pcr_pos_date_1'][y == 0], histtype='stepfilled', bins=bin_edges, alpha=0.4, color='red', density=density)
plt.legend()
plt.show()

In [ ]:
y_pred = data['pcr_pos_date_1'] >480
accuracy = (y_pred == y).mean()
balanced_accuracy = balanced_accuracy_score(y, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print(f"Balanced Accuracy: {balanced_accuracy:.2f}")

In [ ]:
bins = 10
density = False


bins = plt.hist(data['num_vaccines'][y==1], bins=bins, density=density)[1]

plt.hist(data['num_vaccines'][y==1], histtype='step', bins=bins, color='blue', density=density, label='Positive')
plt.hist(data['num_vaccines'][y==1], histtype='stepfilled', bins=bins, alpha=0.4, color='blue', density=density)
plt.hist(data['num_vaccines'][y==0], histtype='step', bins=bins, color='red', density=density, label='Negative')
plt.hist(data['num_vaccines'][y==0], histtype='stepfilled', bins=bins, alpha=0.4, color='red', density=density)
plt.legend()
plt.title("Total number of vaccines")
plt.show()

In [ ]:
((data['num_vaccines']<1.5) == y).mean()

In [ ]:
from sklearn.metrics import balanced_accuracy_score

y_pred = data['num_vaccines'] < 1.5
accuracy = (y_pred == y).mean()
balanced_accuracy = balanced_accuracy_score(y, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print(f"Balanced Accuracy: {balanced_accuracy:.2f}")

In [ ]:
bins = 50
density = False


bins = plt.hist(zvac['index_date'][y==1], bins=bins, density=density)[1]

plt.hist(zvac['index_date'][y==1], histtype='step', bins=bins, color='blue', density=density, label='Positive')
plt.hist(zvac['index_date'][y==1], histtype='stepfilled', bins=bins, alpha=0.4, color='blue', density=density)
plt.hist(zvac['index_date'][y==0], histtype='step', bins=bins, color='red', density=density, label='Negative')
plt.hist(zvac['index_date'][y==0], histtype='stepfilled', bins=bins, alpha=0.4, color='red', density=density)
plt.legend()
plt.show()